<a href="https://colab.research.google.com/github/DANIEL20625/homework/blob/main/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pycaret 
import pycaret

In [1]:
import pandas as pd

In [4]:
BusinessAnalyst = pd.read_csv("drive/MyDrive/BusinessAnalyst.csv")
DataAnalyst = pd.read_csv("drive/MyDrive/DataAnalyst.csv")
DataEngineer = pd.read_csv("drive/MyDrive/DataEngineer.csv")
DataScientist = pd.read_csv("drive/MyDrive/DataScientist.csv")

# 初步讀入資料後可以發現各個資料的欄位數量並不一樣

---

In [5]:
BusinessAnalyst.shape

(4092, 17)

In [6]:
DataAnalyst.shape

(2253, 16)

In [7]:
DataEngineer.shape

(2528, 15)

In [8]:
DataScientist.shape

(3909, 17)

# 欲確認四個資料之間缺少那些欄位，我以BusinessAnalyst 中17個欄位為標準，判斷其他資料與之相差欄位

In [29]:
BusinessAnalyst_col = list(BusinessAnalyst.columns)
DataAnalyst_col = list(DataAnalyst.columns)
DataEngineer_col = list(DataEngineer.columns)
DataScientist_col = list(DataScientist.columns)
col = ['Data','Common_col','Missing_col']
col_missing = pd.DataFrame(columns=col)
col_missing = pd.concat([col_missing,pd.DataFrame([{'Data': 'BusinessAnalyst','Common_col':','.join([x for x in BusinessAnalyst_col if x in BusinessAnalyst_col]), 'Missing_col': ','.join([x for x in BusinessAnalyst_col if x not in BusinessAnalyst_col])}])])
col_missing = pd.concat([col_missing,pd.DataFrame([{'Data': 'DataAnalyst','Common_col':','.join([x for x in BusinessAnalyst_col if x in DataAnalyst_col]), 'Missing_col': ','.join([x for x in BusinessAnalyst_col if x not in DataAnalyst_col])}])])
col_missing = pd.concat([col_missing,pd.DataFrame([{'Data': 'DataEngineer','Common_col':','.join([x for x in BusinessAnalyst_col if x in DataEngineer_col]), 'Missing_col': ','.join([x for x in BusinessAnalyst_col if x not in DataEngineer_col])}])])
col_missing = pd.concat([col_missing,pd.DataFrame([{'Data': 'DataScientist','Common_col':','.join([x for x in BusinessAnalyst_col if x in DataScientist_col]) ,'Missing_col': ','.join([x for x in BusinessAnalyst_col if x not in DataScientist_col])}])])
col_missing

,Data,Common_col,Missing_col
0,BusinessAnalyst,"Unnamed: 0,index,Job Title,Salary Estimate,Job...",
0,DataAnalyst,"Unnamed: 0,Job Title,Salary Estimate,Job Descr...",index
0,DataEngineer,"Job Title,Salary Estimate,Job Description,Rati...","Unnamed: 0,index"
0,DataScientist,"Unnamed: 0,index,Job Title,Salary Estimate,Job...",


**可以發現相差的欄位命名為 [index,Unnamed: 0] 初步判斷與資料內容無太大相關，之後細看合併時可以考慮drop掉** 